In [12]:
# imports de bibliotecas
import numpy as np
import mne
from scipy.signal import welch
import matplotlib.pyplot as plt

from sklearn import svm
#from sklearn import linear_model


In [13]:
def print_graphs(X):
    for i in range(X.shape[1]):
        plt.plot(X[:,i])
    plt.title('Domínio do tempo')
    plt.show()

    for i in range(X.shape[1]):
        plt.psd(X[:,i], Fs=250)
    plt.title('Domínio da frequência')
    plt.show()

    for i in range(X.shape[1]):
        plt.specgram(X[i,:], Fs=250)
    plt.title('Espectrograma')
    plt.show()

In [14]:
# definição do info (MNE)

n_channels = 8
ch_types = ['eeg'] * n_channels
sfreq = 250
ch_names = ["F3", "Fz", "F4", "C3", "Cz", "C4", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_montage("standard_1020")

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [15]:
# carregamento e organização dos dados de: Maria

ga = [
  '../dataset/maria/ia/OpenBCI-RAW-2023-09-28_16-51-25.txt', # aula: 00:08 - 20:02, teste: 20:23 - 23:10
]
tf = [
  '../dataset/maria/tf/OpenBCI-RAW-2023-11-07_13-17-01.txt', # basal: 3:28, teste: 4:28 - 12:29
]
sr = 250                  # taxa de amostragem

ga_ob = [np.loadtxt(txt, delimiter=',', skiprows=5, usecols=range(1, 9)) for txt in ga]
data_ga = ga_ob[0][8*sr:1202*sr,:]
data_ga_test = ga_ob[0][1223*sr:1390*sr,:]

tf_ob = [np.loadtxt(txt, delimiter=',', skiprows=5, usecols=range(1, 9)) for txt in tf]
data_basal = tf_ob[0][208*sr:268*sr,:]
data_tf = tf_ob[0][268*sr:749*sr,:]

# Insulina Ativa
x_ga = mne.io.RawArray(data_ga.T, info)
# Insulina Ativa Teste
x_ga_test = mne.io.RawArray(data_ga_test.T, info)
# Teste final
x_tf = mne.io.RawArray(data_tf.T, info)
# Basal
x_basal = mne.io.RawArray(data_basal.T, info)

# x_ga.set_eeg_reference(ref_channels='average') # CAR
# x_ga_test.set_eeg_reference(ref_channels='average') # CAR
# x_tf.set_eeg_reference(ref_channels='average') # CAR
# x_basal.set_eeg_reference(ref_channels='average') # CAR

X = {
  'ga': x_ga,
  'ga_test': x_ga_test,
  'tf': x_tf,
}

Creating RawArray with float64 data, n_channels=8, n_times=298500
    Range : 0 ... 298499 =      0.000 ...  1193.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=41750
    Range : 0 ... 41749 =      0.000 ...   166.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=120250
    Range : 0 ... 120249 =      0.000 ...   480.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=15000
    Range : 0 ... 14999 =      0.000 ...    59.996 secs
Ready.


In [16]:
# filtragem de todos os dados

for key in X:
    X[key].notch_filter(freqs=60)
    X[key].filter(l_freq=4, h_freq=100)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband e

In [18]:
time = {
  'ga': X['ga'].times[-1],
  'ga_test': X['ga_test'].times[-1],
  'tf': X['tf'].times[-1],
}

sr = 250                  # taxa de amostragem
jump = 5                  # 5s de buffer
size = sr * jump          # quantidade de pontos avalidados
nperseg = 128             # Número de pontos por segmento
noverlap = nperseg // 2   # Quantidade de sobreposição entre segmentos

# Definir os limites das bandas de frequência (em Hz)
theta_band = (4, 8)       # Theta:  4 - 8   Hz
alpha_band = (8, 13)      # Alpha:  8 - 13  Hz
beta_band = (13, 30)      # Beta:  13 - 30  Hz
gamma_band = (30, 100)    # Gamma: 30 - 100 Hz

allBandBuffers = []
bufferBand = []

allBuffers = []
buffer = []
buffer_size = 5
start_time = 0
stop_time = start_time + buffer_size
beta_intervals = {}
data_names = ('ga', 'ga_test', 'tf')

sample = {
  'focus':[],
  'not_focus':[]
}

for k, data in enumerate(X.values()):
  beta_intervals[k] = []
  for i in range(0, int(time[data_names[k]])-buffer_size-1):
    results = [0,0,0,0]
    start_time = i
    stop_time = start_time + buffer_size

    cut = data.get_data(tmin=start_time, tmax=stop_time)
    freqs, psd = welch(cut, fs=sr, nperseg=nperseg, noverlap=noverlap)

    avg = np.average(psd, axis=0)

    # Encontrar os índices correspondentes às frequências de interesse
    theta_idxs = np.where((freqs >= theta_band[0]) & (freqs <= theta_band[1]))[0]
    alpha_idxs = np.where((freqs >= alpha_band[0]) & (freqs <= alpha_band[1]))[0]
    beta_idxs =  np.where((freqs >=  beta_band[0]) & (freqs <=  beta_band[1]))[0]
    gamma_idxs = np.where((freqs >= gamma_band[0]) & (freqs <= gamma_band[1]))[0]

    # Calcular a potência em cada banda de frequência
    bands = [
      np.sum(avg[theta_idxs]), 
      np.sum(avg[alpha_idxs]), 
      np.sum(avg[ beta_idxs]), 
      np.sum(avg[gamma_idxs])
    ]
    results[np.argmax(bands)] += 1

    if len(buffer) == buffer_size:
      del buffer[0]

    buffer.append(results)
    copyBuffer = buffer.copy()
    
    acc = np.array([0,0,0,0])
    for col in copyBuffer:
      acc = acc + np.array(col)

    #fluxo novo
    # if(acc[2] > 2 or acc[3] > 2):
    if(acc[2] > 2 or acc[3] > 4):
      sample['focus'].append(avg)
    else:
      sample['not_focus'].append(avg)
    
    #fluxo antigo
    # if(acc[2] > 2):
    #   beta_intervals[k].append([start_time,stop_time, acc, cut])
    # allBuffers.append(copyBuffer)

In [19]:
# Estimando o ruído de fundo (utilizando o sinal basal)
sample_total = sample['focus'] + sample['focus']
# armazena uma lista com as médias de potência para cada canal
noise_power = []
for channel_data in sample_total:
    # fft_result = np.fft.fft(channel_data)
    # média da potência no intervalo de tempo sem estímulo
    # base_power = np.mean(fft_result)
    # noise_power.append()
    noise_power.append(channel_data)
# média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
print(estimated_background_noise) 
print(noise_power) 

1.858644780668887
[array([5.15162200e-01, 6.28829360e+00, 1.94696877e+01, 1.34663131e+01,
       7.57301095e+00, 4.22085392e+00, 2.80831737e+00, 2.83397391e+00,
       1.83337686e+00, 1.62131435e+00, 1.91614101e+00, 2.21508249e+00,
       2.67465834e+00, 2.19532609e+00, 1.71520067e+00, 1.92076961e+00,
       1.93489950e+00, 2.04456386e+00, 1.90403719e+00, 1.64274312e+00,
       1.62978955e+00, 1.29047861e+00, 1.76911032e+00, 1.78626093e+00,
       1.81520734e+00, 1.50801133e+00, 2.23321996e+00, 1.54486062e+00,
       1.74223806e+00, 1.51340967e+00, 1.21492734e+00, 1.08851349e+00,
       1.42251715e+00, 1.42245453e+00, 1.57800479e+00, 1.18302684e+00,
       1.16042732e+00, 1.36769532e+00, 1.08268438e+00, 7.22134699e-01,
       8.16091358e-01, 8.07538895e-01, 6.58853851e-01, 6.21789099e-01,
       7.41494783e-01, 7.95898231e-01, 5.47121370e-01, 4.30933350e-01,
       4.44479072e-01, 4.32897915e-01, 4.05996626e-01, 3.78647427e-01,
       3.46446246e-01, 2.97625201e-01, 2.86616509e-01, 2.2

In [20]:
# agora vamos adaptar ambas características 
# aplicando para o nosso sinal de interesse

# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
target_amplitudes_adjusted = []
for d in sample['focus']:
  target_amplitudes_adjusted.append(d - estimated_background_noise)

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)

[[        nan  3.77172901  9.76588679 ...         nan         nan
          nan]
 [        nan  2.2771266   8.67084346 ...         nan         nan
          nan]
 [        nan  0.53514098  6.89656067 ...         nan         nan
          nan]
 ...
 [        nan -1.31343599  6.18477748 ...         nan         nan
          nan]
 [        nan         nan  2.05597479 ...         nan         nan
          nan]
 [        nan         nan -0.57065966 ...         nan         nan
          nan]]
(1016, 65)
[[109.42188085          nan          nan ... 110.82523984 110.83089396
  110.83142073]
 [109.84834817          nan          nan ... 110.82559922 110.83092265
  110.83141281]
 [109.66541235          nan          nan ... 110.82566092 110.83106115
  110.83147203]
 ...
 [110.16281595          nan          nan ... 110.83029431 110.83143538
  110.8315169 ]
 [110.15297016  81.38959978          nan ... 110.83055473 110.8314777
  110.83151795]
 [110.34981494 105.79519203          nan ... 110.83054011 

/tmp/ipykernel_5518/1397904830.py:10: RuntimeWarning: invalid value encountered in log10
  narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
/tmp/ipykernel_5518/1397904830.py:15: RuntimeWarning: invalid value encountered in log10
  wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)


In [21]:
focus = sample['focus']
not_focus = sample['not_focus']
print(len(focus))
print(len(not_focus))

f_divider = int(len(focus) * 0.7)
focus_train = focus[:f_divider] # 70%
focus_test = focus[f_divider:]  # 30%

nf_divider = int(len(not_focus) * 0.7)
not_focus_train = not_focus[:nf_divider] # 70%
not_focus_test = not_focus[nf_divider:]  # 30%


focus_labels = [0] * f_divider
not_focus_labels = [1] * nf_divider

# train_data = focus_train + not_focus_train
# train_label = focus_labels + not_focus_labels
train_data = not_focus_train + focus_train 
train_label = not_focus_labels + focus_labels

train_data # dados 
train_label # labels
clf = svm.SVC(kernel='linear')
clf.fit(train_data, train_label)

1016
805


SVC(kernel='linear')

In [22]:
print("focus_train:",len(focus_train))
print("focus_test:",len(focus_test))
print("not_focus_train:",len(not_focus_train))
print("not_focus_test:",len(not_focus_test))

focus_train: 711
focus_test: 305
not_focus_train: 563
not_focus_test: 242


In [23]:
predict_focus = clf.predict(focus_test)
focus_correct = 0
for data in predict_focus:
  if data == 0:
    focus_correct+=1

not_focus_correct = 0
predict_not_focus = clf.predict(not_focus_test)
for data in predict_not_focus:
  if data == 1:
    not_focus_correct+=1


focus_percent = (focus_correct / len(focus_test))*100
not_focus_percent = (not_focus_correct / len(not_focus_test))*100
total_focus_percent = ((not_focus_correct+focus_correct) / (len(focus_test)+len(not_focus_test)))*100

print(f"FOCUS: {focus_correct}/{len(focus_test)} ({focus_percent}%) correct")
print(f"NOT FOCUS: {not_focus_correct}/{len(not_focus_test)} ({not_focus_percent}%) correct")
print(f"TOTAL: {not_focus_correct+focus_correct}/{len(focus_test)+len(not_focus_test)}({total_focus_percent}%) correct")

FOCUS: 299/305 (98.0327868852459%) correct
NOT FOCUS: 109/242 (45.04132231404959%) correct
TOTAL: 408/547(74.58866544789763%) correct


In [24]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=19)
knn.fit(train_data, train_label)


knn_focus = knn.predict(focus_test)
knn_not_focus = knn.predict(not_focus_test)

knn_focus_correct = 0
for data in knn_focus:
  if data == 0:
    knn_focus_correct+=1

knn_not_focus_correct = 0
for data in knn_not_focus:
  if data == 1:
    knn_not_focus_correct+=1


knn_f_percent = (knn_focus_correct/len(focus_test)) * 100
knn_nf_percent = (knn_not_focus_correct/len(not_focus_test)) * 100
print(f"FOCUS: {knn_focus_correct}/{len(focus_test)} ({knn_f_percent}%) correct")
print(f"NOT FOCUS: {knn_not_focus_correct}/{len(not_focus_test)} ({knn_nf_percent}%)  correct")


FOCUS: 295/305 (96.72131147540983%) correct
NOT FOCUS: 152/242 (62.8099173553719%)  correct
